<a href="https://colab.research.google.com/github/kjmobile/lb/blob/main/4_KNN_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [1]:
import pandas as pd
import numpy as np
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


In [2]:
# import data from mySQL database  using the following info host: kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com, id cob, password 1212, db='data', port 3306

import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com',
                             user='',
                             password='',
                             db='data',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    # Create a cursor object
    with connection.cursor() as cursor:
        # Execute the SQL query to fetch all data from the 'data' table
        cursor.execute("SELECT * FROM data.fish_k")

        # Fetch all the results
        results = cursor.fetchall()

        # Convert the results to a Pandas DataFrame
        fish_k = pd.DataFrame(results)

finally:
    # Close the connection
    connection.close()

# Print the DataFrame
fish_k['species'] = [1 if s=='Swedish' else 0 for s in fish_k["species"]]
print(fish_k)

OperationalError: (1045, "Access denied for user 'root'@'35.231.130.29' (using password: NO)")

In [ ]:
fish_X= fish_k.iloc[:,:2].to_numpy()
fish_y=fish_k['species'].to_numpy()

In [ ]:
fish_X.shape

In [ ]:
fish_y.shape

#Preprocessing :  Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_input, test_input, train_target, test_target = train_test_split(
    fish_X, fish_y, random_state=42)

In [ ]:
print(train_input.shape, test_input.shape)

In [ ]:
print(train_target.shape, test_target.shape)

In [ ]:
print(test_target)

In [ ]:
kn.predict(fish_k.iloc[:,:2])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(train_input[:, 0], train_input[:, 1])
plt.scatter(test_input[:, 0], test_input[:, 1])
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(fish_k['length'], fish_k['weight'],
            c=["C3" if c==1 else "C4" for c in kn.predict(fish_k.iloc[:,:2])])

# Test One Fish!

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(train_input, train_target)
kn.score(test_input, test_target)

In [ ]:
kn?

In [ ]:
print(kn.predict([[25, 150]]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(train_input[:,0], train_input[:,1])
plt.scatter(25, 150, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
distances, indexes = kn.kneighbors([[25, 150]])

In [ ]:
train_input

In [ ]:
plt.scatter(train_input[:,0], train_input[:,1])
plt.scatter(25, 150, marker='^')
plt.scatter(train_input[indexes,0], train_input[indexes,1], marker='D')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
print(train_input[indexes])

In [ ]:
print(train_target[indexes])

In [ ]:
print(distances)

# Plot with comparable xlim (i.e., same scope of length as weight)

In [ ]:
plt.scatter(train_input[:,0], train_input[:,1])
plt.scatter(25, 150, marker='^')
plt.scatter(train_input[indexes,0], train_input[indexes,1], marker='D')
plt.xlim((0, 1000))
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

#With this shape data, the length of the fish (x-axis) would not affect finding the nearest neighbors.
#Only the weight of the fish (y-axis) would be considered.
#--> Standardization of "scale" is necessary in training "distance"-based-algorithms.

# Prepropcessing: Standardization using z scale

In [ ]:
print(train_input[:3,:])

In [ ]:
np.mean?


In [ ]:
mean= np.mean(train_input)
std =np.std(train_input)

In [ ]:
train_scaled = (train_input - mean) / std
print(train_scaled[:3,:])

# Train with scaled train set;
  - Now there is another problem: the test data ([[25,150]]) should also be standardized!

In [ ]:
# the not-yet-standardized test data([[25,150]]) distorts the plot
plt.scatter(train_scaled[:,0], train_scaled[:,1])
plt.scatter(25, 150, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
new = ([25, 150] - mean) / std

In [ ]:
plt.scatter(train_scaled[:,0], train_scaled[:,1])
plt.scatter(new[0], new[1], marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
kn.fit(train_scaled, train_target)

In [ ]:
test_scaled = (test_input - mean) / std

In [ ]:
print(kn.predict([new]))

In [ ]:
distances, indexes = kn.kneighbors([new])

In [ ]:
plt.scatter(train_scaled[:,0], train_scaled[:,1])
plt.scatter(new[0], new[1], marker='^')
plt.scatter(train_scaled[indexes,0], train_scaled[indexes,1], marker='D')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

# Model's accuracy score - Use test dataset

In [ ]:
# Now the model 'correctly' classifies the test fish as caegory 1 (i.e., McFish)
# This is a convenience method same as accuracy_score() shown below.
kn.score(test_scaled, test_target)

# Performance Evaluation: Confusion Matrix and the Four Classification Peformance Metrics   

- **Accuracy** : assesses the overall correctness of the model (obtainable by ``kn.score()``) : $\frac{TP + TN}{TP + TN + FP + FN}$
- **Specificity**: measures the ability to correctly identify negatives :  $\frac{TN}{TN + FP}$
- **Precision**: focuses on the correctness of positive predictions: $\frac{TP}{TP + FP}$
- **Recall** (Sensitivity): measures ability to identify all relevant instances: $\frac{TP}{TP + FN}$

In [3]:
confusion_matrix?

Object `confusion_matrix` not found.


In [ ]:
from sklearn.metrics import confusion_matrix
# Get the confusion matrix
cm = confusion_matrix(test_target, kn.predict(test_scaled))
# Convert the confusion matrix to a DataFrame
cm_df = pd.DataFrame(cm, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])
# Print the DataFrame
print(cm_df)



In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Four classification metrics
accuracy = accuracy_score(test_target, kn.predict(test_scaled))
specificity = recall_score(test_target, kn.predict(test_scaled), pos_label=0)
precision = precision_score(test_target, kn.predict(test_scaled), pos_label=1)
recall = recall_score(test_target, kn.predict(test_scaled), pos_label=1)

print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Precision:", precision)
print("Recall:", recall)
